<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/jina/220314_3_sub3_roberta_large_devO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 3.8 MB 23.1 MB/s 
     |████████████████████████████████| 1.2 MB 65.0 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 6.5 MB 36.1 MB/s 
     |████████████████████████████████| 596 kB 73.7 MB/s 
     |████████████████████████████████| 895 kB 74.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=1909f9afc8c0bbe0ba7b25acc579879192271e162caae08b139589aa97d75ba9
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import argparse
import logging
import math
import os
import random
from datetime import datetime
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, LoggingHandler, models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import json
from sentence_transformers.readers import InputExample

In [5]:
# Parse command line arguments
parser = argparse.ArgumentParser()
parser.add_argument("--model_name_or_path", type=str, default = "klue/roberta-large")
parser.add_argument("--max_seq_length", type=int, default=128)
parser.add_argument("--batch_size", type=int, default=8)
parser.add_argument("--num_epochs", type=int, default=5)
parser.add_argument("--output_dir", type=str, default="output")
parser.add_argument("--output_prefix", type=str, default="kor_sts_")
parser.add_argument("--seed", type=int, default=777)
args = parser.parse_args("")

In [6]:
# Fix random seed
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# Configure logger
logging.basicConfig(
    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO, handlers=[LoggingHandler()]
)

# Read the dataset
model_save_path = os.path.join(
    args.output_dir,
    args.output_prefix + args.model_name_or_path.replace("/", "-") + '-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [7]:
# Define SentenceTransformer model
word_embedding_model = models.Transformer(args.model_name_or_path, max_seq_length=args.max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device = 'cuda')

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [9]:
def load_kor_sts_samples(filename):
    samples = []
    with open(filename,'r') as f:
        json_data = json.load(f)
    for row in json_data:
        score = float(row['labels']['label']) / 5.0  # Normalize score to range 0 ... 1
        samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
    return samples

In [10]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [11]:
%cd /content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1

/content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1


In [25]:
# Read the dataset
logging.info("Read KorSTS train/dev dataset")

train_file = '/content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1/klue-sts-v1.1_train.json'
train_samples = load_kor_sts_samples(train_file)

valid_file = './klue-sts-v1.1_dev.json'
valid_samples = load_kor_sts_samples(valid_file)

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=args.batch_size)
valid_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_samples, batch_size=args.batch_size,
                                                                 name='sts-valid')

2022-03-14 14:29:48 - Read KorSTS train/dev dataset


In [26]:
train_loss = losses.CosineSimilarityLoss(model=model)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * args.num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2022-03-14 14:29:54 - Warmup-steps: 730


In [27]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=valid_evaluator,
          epochs=args.num_epochs,
          show_progress_bar = True,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1459 [00:00<?, ?it/s]

2022-03-14 14:35:01 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 0 after 1000 steps:
2022-03-14 14:35:04 - Cosine-Similarity :	Pearson: 0.8731	Spearman: 0.8706
2022-03-14 14:35:04 - Manhattan-Distance:	Pearson: 0.8642	Spearman: 0.8668
2022-03-14 14:35:04 - Euclidean-Distance:	Pearson: 0.8646	Spearman: 0.8672
2022-03-14 14:35:04 - Dot-Product-Similarity:	Pearson: 0.8179	Spearman: 0.8133
2022-03-14 14:35:04 - Save model to output/kor_sts_klue-roberta-large-2022-03-14_13-43-19
2022-03-14 14:37:28 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 0:
2022-03-14 14:37:32 - Cosine-Similarity :	Pearson: 0.8771	Spearman: 0.8806
2022-03-14 14:37:32 - Manhattan-Distance:	Pearson: 0.8723	Spearman: 0.8803
2022-03-14 14:37:32 - Euclidean-Distance:	Pearson: 0.8726	Spearman: 0.8799
2022-03-14 14:37:32 - Dot-Product-Similarity:	Pearson: 0.8269	Spearman: 0.8281
2022-03-14 14:37:32 - Save model to output/kor_sts_klue-roberta-large-

Iteration:   0%|          | 0/1459 [00:00<?, ?it/s]

...

2022-03-14 14:50:18 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 2 after 1000 steps:
2022-03-14 14:50:22 - Cosine-Similarity :	Pearson: 0.8916	Spearman: 0.8910
2022-03-14 14:50:22 - Manhattan-Distance:	Pearson: 0.8708	Spearman: 0.8742
2022-03-14 14:50:22 - Euclidean-Distance:	Pearson: 0.8718	Spearman: 0.8759
2022-03-14 14:50:22 - Dot-Product-Similarity:	Pearson: 0.8191	Spearman: 0.8166
2022-03-14 14:50:22 - Save model to output/kor_sts_klue-roberta-large-2022-03-14_13-43-19
2022-03-14 14:52:45 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 2:
2022-03-14 14:52:49 - Cosine-Similarity :	Pearson: 0.8977	Spearman: 0.8979
2022-03-14 14:52:49 - Manhattan-Distance:	Pearson: 0.8849	Spearman: 0.8871
2022-03-14 14:52:49 - Euclidean-Distance:	Pearson: 0.8854	Spearman: 0.8878
2022-03-14 14:52:49 - Dot-Product-Similarity:	Pearson: 0.8372	Spearman: 0.8344
2022-03-14 14:52:49 - Save model to output/kor_sts_klue-roberta-large-

Iteration:   0%|          | 0/1459 [00:00<?, ?it/s]

2022-03-14 14:57:55 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 3 after 1000 steps:
2022-03-14 14:57:59 - Cosine-Similarity :	Pearson: 0.8899	Spearman: 0.8902
2022-03-14 14:57:59 - Manhattan-Distance:	Pearson: 0.8756	Spearman: 0.8800
2022-03-14 14:57:59 - Euclidean-Distance:	Pearson: 0.8756	Spearman: 0.8805
2022-03-14 14:57:59 - Dot-Product-Similarity:	Pearson: 0.8161	Spearman: 0.8156
2022-03-14 15:00:16 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 3:
2022-03-14 15:00:19 - Cosine-Similarity :	Pearson: 0.8914	Spearman: 0.8910
2022-03-14 15:00:19 - Manhattan-Distance:	Pearson: 0.8782	Spearman: 0.8831
2022-03-14 15:00:19 - Euclidean-Distance:	Pearson: 0.8780	Spearman: 0.8830
2022-03-14 15:00:19 - Dot-Product-Similarity:	Pearson: 0.8246	Spearman: 0.8235


Iteration:   0%|          | 0/1459 [00:00<?, ?it/s]

2022-03-14 15:05:20 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 4 after 1000 steps:
2022-03-14 15:05:23 - Cosine-Similarity :	Pearson: 0.8934	Spearman: 0.8934
2022-03-14 15:05:23 - Manhattan-Distance:	Pearson: 0.8790	Spearman: 0.8845
2022-03-14 15:05:23 - Euclidean-Distance:	Pearson: 0.8788	Spearman: 0.8844
2022-03-14 15:05:23 - Dot-Product-Similarity:	Pearson: 0.8250	Spearman: 0.8232
2022-03-14 15:07:42 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 4:
2022-03-14 15:07:46 - Cosine-Similarity :	Pearson: 0.8932	Spearman: 0.8933
2022-03-14 15:07:46 - Manhattan-Distance:	Pearson: 0.8793	Spearman: 0.8846
2022-03-14 15:07:46 - Euclidean-Distance:	Pearson: 0.8790	Spearman: 0.8845
2022-03-14 15:07:46 - Dot-Product-Similarity:	Pearson: 0.8260	Spearman: 0.8243


In [20]:
!pip install datasets

     |████████████████████████████████| 312 kB 3.9 MB/s 
     |████████████████████████████████| 1.1 MB 54.4 MB/s 
     |████████████████████████████████| 212 kB 80.3 MB/s 
     |████████████████████████████████| 134 kB 71.9 MB/s 
     |████████████████████████████████| 127 kB 76.7 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 78.9 MB/s 
     |████████████████████████████████| 271 kB 76.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [21]:
from torch.utils.data import DataLoader
from datasets import load_dataset

In [28]:
logging.info("Read KorNLU benchmark test dataset")
test_file_2 = load_dataset("kor_nlu", "sts")
test_samples_2=[]
for example in test_file_2["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples_2.append(inp_example)

test_evaluator_2 = EmbeddingSimilarityEvaluator.from_input_examples(test_samples_2, name='sts-test')
test_evaluator_2(model, output_path=model_save_path)

2022-03-14 15:07:54 - Read KorNLU benchmark test dataset
2022-03-14 15:07:56 - Reusing dataset kor_nlu (/root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8)


  0%|          | 0/3 [00:00<?, ?it/s]

2022-03-14 15:07:56 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-03-14 15:08:03 - Cosine-Similarity :	Pearson: 0.7521	Spearman: 0.7441
2022-03-14 15:08:03 - Manhattan-Distance:	Pearson: 0.7085	Spearman: 0.7151
2022-03-14 15:08:03 - Euclidean-Distance:	Pearson: 0.7078	Spearman: 0.7142
2022-03-14 15:08:03 - Dot-Product-Similarity:	Pearson: 0.7276	Spearman: 0.7235


0.7440933403628609